In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
a = pd.read_csv("../기상청공모전/data/derivate_data.csv")

In [5]:
a1 = a.iloc[:,1:]

In [6]:
a1 = a1[a1["cog"] != -99.9]
a1 = a1[a1["hdg"] != -99.9]
a1 = a1[a1["hdg"] != -99]

In [2]:
!pip install catboost

     ------------------------------------- 101.0/101.0 MB 10.2 MB/s eta 0:00:00



[notice] A new release of pip available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# CatBoost
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

X = a1.iloc[:, 2:]  # 세번째 열부터 마지막 열까지 입력값
y = a1.iloc[:, 0]  # 첫번째 열이 정답열

# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CatBoost 모델 학습
model = CatBoostClassifier(iterations=100, eval_metric='Logloss')
model.fit(X_train, y_train, verbose=False)

# 테스트 데이터 예측
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]  # 클래스 1에 대한 예측 확률 추출

# 이진 분류에서의 예측 결과 변환
y_pred_binary = [1 if y >= 0.5 else 0 for y in y_pred_proba]

# 정확도 평가
accuracy = accuracy_score(y_test, y_pred_binary)
print("정확도:", accuracy)

# F1 점수 계산
f1 = f1_score(y_test, y_pred_binary)
print("F1 점수:", f1)

정확도: 0.991244027445445
F1 점수: 0.7702060221870047


In [8]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix 계산
cm = confusion_matrix(y_test, y_pred_binary)

# CSI 점수 계산
tn, fp, fn, tp = cm.ravel()
csi = tp / (tp + fp + fn)
print("CSI 점수:", csi)

CSI 점수: 0.6262886597938144
